In [23]:
from census import Census
from us import states
import pandas
 
def get_state(state_county):
    return state_county.split(", ")[1]
 
def get_county(state_county):
    return state_county.split(", ")[0]
 
c = Census("f88810dbb816ce76eb26fc06cb5d34c2ea1f96ad")
data = pandas.DataFrame(c.acs5.get(('B24031_001E', 'B24031_002E', 'B24031_003E', 'B24031_004E', 'B24031_005E', 'B24031_006E', 'B24031_007E', 'B24031_008E', 'B24031_009E', 'B24031_010E', 'B24031_011E', 'B24031_012E', 'B24031_013E', 'B24031_014E', 'B24031_015E', 'B24031_016E', 'B24031_017E', 'B24031_018E', 'B24031_019E', 'B24031_020E', 'B24031_021E', 'B24031_022E', 'B24031_023E', 'B24031_024E', 'B24031_025E', 'B24031_026E', 'B24031_027E'), {'for': 'county:*'}))
data.to_csv("census_data4.csv")

In [24]:
data

,B24031_001E,B24031_002E,B24031_003E,B24031_004E,B24031_005E,B24031_006E,B24031_007E,B24031_008E,B24031_009E,B24031_010E,...,B24031_020E,B24031_021E,B24031_022E,B24031_023E,B24031_024E,B24031_025E,B24031_026E,B24031_027E,state,county
0,34148.0,28750.0,26250.0,74643.0,40313.0,22083.0,18750.0,24609.0,32361.0,31806.0,...,40962.0,55000.0,32500.0,20625.0,60625.0,17500.0,43750.0,61667.0,02,195
1,22562.0,94375.0,12917.0,123681.0,25556.0,37466.0,-666666666.0,12273.0,18870.0,19031.0,...,26117.0,27083.0,23289.0,6979.0,5625.0,24219.0,8750.0,20556.0,02,158
2,36531.0,33680.0,27838.0,64502.0,37268.0,51621.0,46553.0,25877.0,42886.0,40234.0,...,37882.0,37362.0,38489.0,18524.0,24248.0,17287.0,25239.0,53295.0,04,013
3,31445.0,59949.0,31200.0,62694.0,31126.0,25822.0,29821.0,23089.0,31783.0,31250.0,...,31306.0,34563.0,30619.0,12118.0,17898.0,11574.0,26622.0,41527.0,04,009
4,26688.0,21302.0,14667.0,59167.0,28793.0,40820.0,29936.0,19872.0,32183.0,31863.0,...,29167.0,31898.0,26941.0,14751.0,24974.0,13617.0,20477.0,38977.0,04,023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,30513.0,50464.0,32012.0,80096.0,35625.0,34710.0,39533.0,20403.0,55700.0,54675.0,...,35003.0,40902.0,30938.0,14385.0,17639.0,14318.0,21625.0,40497.0,48,005
3216,25864.0,31250.0,25588.0,48942.0,40341.0,-666666666.0,-666666666.0,21167.0,22031.0,-666666666.0,...,25927.0,25625.0,25948.0,10521.0,-666666666.0,10104.0,9474.0,29583.0,48,101
3217,34657.0,46795.0,19777.0,81083.0,37947.0,53233.0,53785.0,22772.0,35723.0,31083.0,...,37028.0,36993.0,37047.0,14194.0,20347.0,13967.0,30766.0,53304.0,48,209
3218,47666.0,79406.0,35320.0,90375.0,48352.0,76018.0,59323.0,21032.0,60559.0,56071.0,...,50493.0,48263.0,52787.0,15478.0,20518.0,14998.0,30600.0,49894.0,48,039
